In [ ]:
from pathlib import Path

def find_project_root(name_substring: str = "riseholme",
                      required_subpath: Path = Path("Data") / "Normal" / "Ripe",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst passt (Name + required_subpath).
      2. Prüft alle direkten Unterverzeichnisse (Geschwister), ob dort
         name_substring im Namen ist und required_subpath existiert.
    Gibt das gefundene Verzeichnis (Path) zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Ist p selbst unser Projekt-Root?
        if name_substring.lower() in p.name.lower() and (p / required_subpath).is_dir():
            return p

        # 2) Sonst durchsuche alle Unterordner von p nach unserem Projekt-Root
        for child in p.iterdir():
            if (child.is_dir()
                and name_substring.lower() in child.name.lower()
                and (child / required_subpath).is_dir()):
                return child

    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält "
                            f"und '{name_substring}' im Namen trägt (Beginn: {start})")

# Verwende die neue Funktion
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# Jetzt kannst du weitermachen:
image_dir = BASE_DIR / 'Data' / 'Normal' / 'Ripe'
label_dir = BASE_DIR / 'labels' / 'Ripe'
label_dir.mkdir(parents=True, exist_ok=True)

class_id = 0  # Klasse 0 = "ripe"
count = 0

for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")


datensatz splitten

In [5]:
from pathlib import Path
import sys
!{sys.executable} -m pip install scikit-learn

from sklearn.model_selection import train_test_split
import shutil


def find_project_root(required_subpath: Path = Path("Data") / "Normal" / "Ripe",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst required_subpath enthält.
      2. Durchsucht alle Unterordner jedes Eltern-Verzeichnisses nach required_subpath.
    Gibt das erste gefundene Verzeichnis zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Passt p selbst?
        if (p / required_subpath).is_dir():
            return p
        # 2) Oder eine seiner Unterordner?
        for child in p.iterdir():
            if child.is_dir() and (child / required_subpath).is_dir():
                return child
    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält (Beginn: {start})")

# 1) Projekt-Root ermitteln
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# 2) Quell- und Ziel-Pfade definieren
IMG_SRC   = BASE_DIR / 'Data'    / 'Normal' / 'Ripe'
LBL_SRC   = BASE_DIR / 'labels'  / 'Ripe'

IMG_TRAIN = BASE_DIR / 'images'  / 'train'
IMG_VAL   = BASE_DIR / 'images'  / 'val'
LBL_TRAIN = BASE_DIR / 'labels'  / 'train'
LBL_VAL   = BASE_DIR / 'labels'  / 'val'

# 3) Zielordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# 4) Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png', '.jpg', '.jpeg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# 5) Kopieren
for f in train:
    # Bild-Extension automatisch prüfen
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_TRAIN / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_TRAIN / src_lbl.name)

for f in val:
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_VAL / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_VAL / src_lbl.name)

print(f"✅ Train: {len(train)} | Val: {len(val)} Dateien kopiert")


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.1 MB 7.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.1 MB 7.3 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.1 MB 7.3 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 7.3 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.1 MB 6.9 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.1 MB 7.3 MB/s eta 0:00:01
   -------------------------------------- - 10.7/11.1 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/12.6 MB 7.4 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/12.6 MB 7.3 MB/s eta 0:00:02
   -------------- ------------------------- 4.5/12.6 MB 7.3 MB/s eta 0:00:02
   --------

In [ ]:
import sys
!{sys.executable} -m pip install ultralytics

from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=50,                         # je nach Bedarf , 25 reichen (1h)
    imgsz=640,
    batch=16,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='ripe_only'                   # Lauf-Name
)


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.1 MB 7.3 MB/s eta 0:00:01
   -------------- ------------------------- 2.9/8.1 MB 7.2 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.1 MB 7.2 MB/s eta 0:00:01
   ----------------------------- ---------- 6.0/8.1 MB 7.2 MB/s eta 0:00:01
   ------------------------------------ --- 7.3/8.1 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------------------- ---------------- 1.3/2.2 MB 7.2 MB/s eta 0:00:01
   ----------------------------

100%|██████████| 755k/755k [00:00<00:00, 7.30MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access  (ping: 0.20.0 ms, read: 2.00.8 MB/s, size: 12.1 KB)


train: Scanning C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\train... 369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<00:00, 484.80it/s]


...

                   all         93         93      0.999          1      0.995      0.995



50 epochs completed in 1.864 hours.
Optimizer stripped from runs\train\ripe_only\weights\last.pt, 6.2MB
Optimizer stripped from runs\train\ripe_only\weights\best.pt, 6.2MB

Validating runs\train\ripe_only\weights\best.pt...
Ultralytics 8.3.120  Python-3.13.3 torch-2.7.0+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:20<00:00,  6.84s/it]


                   all         93         93      0.999          1      0.995      0.995
Speed: 1.8ms preprocess, 210.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\train\ripe_only


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024AB7F20830>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480